In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold

from mpl_toolkits import mplot3d
%matplotlib inline
import matplotlib.pyplot as plt

from ipywidgets import interact
import ipywidgets as widgets
from ipywidgets import fixed

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
features = ["Pclass", "Sex", "SibSp", "Parch"]

exp_data = pd.read_csv("./train.csv")
X = pd.get_dummies(exp_data[features])
y = exp_data['Survived']
exp_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [3]:
submition_data = pd.read_csv("./test.csv")
submition_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
test_per = 0.1

X_train_val, X_test, y_train_val, y_test \
    = train_test_split(X, y, test_size=test_per, random_state=1)

print('total experimental data', X.shape[0])
print('training and validation num rows:', X_train_val.shape[0])
print('test num rows:', X_test.shape[0])

total experimental data 891
training and validation num rows: 801
test num rows: 90


In [6]:
from sklearn.ensemble import RandomForestClassifier

num_folds = 10
folds = np.array_split(range(X_train_val.shape[0]), num_folds)

avg_errors = []
num_est_param = []
max_depth_param = []

for num_estimators in [10, 20, 30, 50, 70, 100]:
    for max_depth in range(1,20,2):
        errors = []
        for val_fold in range(len(folds)):
            val_indicies = folds[val_fold]
            X_val = X_train_val.iloc[val_indicies]
            y_val = y_train_val.iloc[val_indicies]

            train_indicies = sum([list(folds[fold_ind]) for fold_ind in range(len(folds))\
                                     if fold_ind != val_fold], [])

            X_train = X_train_val.iloc[train_indicies]
            y_train = y_train_val.iloc[train_indicies]

            model = RandomForestClassifier(n_estimators=num_estimators, max_depth=max_depth, random_state=1)
            model.fit(X_train, y_train)
            predictions = model.predict(X_val)
            assert(len(y_val) == len(predictions))
            errors.append(sum([np.abs(y_val.iloc[i] - predictions[i]) for i in range(len(y_val))])/len(y_val))
        num_est_param.append(num_estimators)
        max_depth_param.append(max_depth)
        avg_errors.append(np.average(errors))
        
    print("Completed Iteration")
    print("num_estimators: ", num_estimators)
    print("Best accuracy so far:", 1 - min(avg_errors))
    print("==========================================")

min_index = np.argmin(avg_errors)
print("The highest average accuracy was produced with parameters,")
print("num_estimators: ", num_est_param[min_index])
print("max_depth: ", max_depth_param[min_index])
print("Average Accuracy:", 1 - avg_errors[min_index])
            

Completed Iteration
num_estimators:  10
Best accuracy so far: 0.8039814814814814
Completed Iteration
num_estimators:  20
Best accuracy so far: 0.8052314814814815
Completed Iteration
num_estimators:  30
Best accuracy so far: 0.8064814814814815
Completed Iteration
num_estimators:  50
Best accuracy so far: 0.8102314814814815
Completed Iteration
num_estimators:  70
Best accuracy so far: 0.8114814814814815
Completed Iteration
num_estimators:  100
Best accuracy so far: 0.8114814814814815
The highest average accuracy was produced with parameters,
num_estimators:  70
max_depth:  3
Average Accuracy: 0.8114814814814815


In [8]:
opts = list(set(num_est_param))
opts.sort()
slider = widgets.SelectionSlider(description='estimators', options=opts)

def display(num_estimators):
    indicies = [i for i, val in enumerate(num_est_param) if val == num_estimators]
    depth_sel = list(map(lambda ind: max_depth_param[ind], indicies))
    errs_sel = list(map(lambda ind: avg_errors[ind], indicies))
    plt.xlabel('max depth')
    plt.ylabel('error')
    plt.plot(depth_sel, errs_sel)
    
interact(display, num_estimators=slider)

interactive(children=(SelectionSlider(description='estimators', options=(10, 20, 30, 50, 70, 100), value=10), …

<function __main__.display(num_estimators)>